# How to PyPSA

In [1]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.font_manager

In [11]:
datadir = '/home/users/train061/group_folder/data/'
model = 'ICHEC'
period = '1970-1976'

In [12]:
network_d = datadir+'schlott_material/{}/{}'.format(model, period)

network = pypsa.Network(network_d)

Importing PyPSA from older version of PyPSA than current version 0.17.1.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

/home/users/train061/envs/pypsa-eur_2/lib/python3.8/site-packages/pypsa/io.py:769: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  pnl[attr] = pnl[attr].reindex(columns=df.index | columns, fill_value=default)
/home/users/train061/envs/pypsa-eur_2/lib/python3.8/site-packages/pypsa/io.py:771: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  pnl[attr] = pnl[attr].reindex(columns=(pnl[attr].columns | columns))
INFO:pypsa.io:Imported network 1970-1976 has buses, carriers, generators, links, loads, storage_units


## Features
Recommendation: Try to investigate measures which can be expressed as a function of the climate period (and potentially the node/carriers). Then, time series can be plotted and compared, similar periods can be identified, measures can be plotted against each other (scatter plots) to identify dependencies.

### Met Input Data

In [29]:
### average capacity factors for resources wind (onwind + offwind), solar PV (solar) and runoff river (ror)

n_timesteps = len(network.snapshots)
n_generators_per_carrier = pd.Series({carrier: len(network.generators.index[network.generators.carrier==carrier])
                                     for carrier in ['onwind', 'offwind', 'solar', 'ror']})

avg_capacity_factor = network.generators_t.p_max_pu.mean() # per generator

avg_capacity_factor_per_carrier = (network.generators_t.p_max_pu.groupby(network.generators.carrier, axis=1).sum().sum()/
                                   (n_timesteps*n_generators_per_carrier)) # per carrier

avg_capacity_factor_per_carrier_and_node = network.generators_t.p_max_pu.groupby([network.generators.bus, network.generators.carrier], 
                                                                                 axis=1).sum().sum()/n_timesteps # per carrier and bus

In [113]:
### low-generation events (minimum power availability from VRES over a period of 2-4 days, see Kies_etal)
vregens = network.generators.index[network.generators.carrier.isin(['onwind','offwind','solar'])]
available_p = network.generators_t.p_max_pu[vregens].sum(axis=1)

minimum_p_available_pu = pd.Series({n_days: (available_p.rolling('{}D'.format(n_days)).sum()/(len(vregens)*n_days*8)).min()
                                    for n_days in range(2,5)})

In [16]:
### complementarity as described in .pptx in group folder
### for further complementarity indices see Jurasz_etal

complementarity_seasonal = pd.read_csv(datadir+'complementarity/{}/{}/seasonal.csv'.format(model, period), header=0, index_col=0) # only NaNs => Bruno

In [ ]:
### ramp rates (see Kies_etal)
### correlation lengths (see Schlott_etal)

### Simulation Outputs

In [62]:
### capacity/generation shares
cap_share = network.generators.p_nom_opt.groupby(network.generators.carrier).sum() # per carrier

gen_share = network.generators_t.p.groupby(network.generators.carrier, axis=1).sum().sum() # per carrier
gen_share_per_node = network.generators_t.p.groupby([network.generators.bus, network.generators.carrier], axis=1).sum().sum() # per carrier and bus

In [52]:
### levelized cost of electricity
investments = network.generators.p_nom_opt * network.generators.capital_cost
operation = network.generators_t.p.sum() * network.generators.marginal_cost

generation_per_carrier = network.generators_t.p.groupby(network.generators.carrier,axis=1).sum()
operation_per_carrier = operation.groupby(network.generators.carrier).sum()
investments_per_carrier = investments.groupby(network.generators.carrier).sum()

generation_per_carrier_and_node = network.generators_t.p.groupby([network.generators.bus, network.generators.carrier],axis=1).sum()
operation_per_carrier_and_node = operation.groupby([network.generators.bus, network.generators.carrier]).sum()
investments_per_carrier_and_node = investments.groupby([network.generators.bus, network.generators.carrier]).sum()

lcoe = (investments.sum() + operation.sum()) / network.loads_t.p_set.sum().sum() # for whole system
lcoe_per_carrier = (investments_per_carrier.sum() + operation_per_carrier.sum()) / generation_per_carrier.sum() # per carrier
lcoe_per_carrier_and_node = ((investments_per_carrier_and_node.sum() + operation_per_carrier_and_node.sum()) / 
                    generation_per_carrier_and_node.sum()) # per carrier and bus